<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Tutorial on generating an explanation for an image-based model on Watson OpenScale (IBM Cloud Pak for Data 2.5)

This notebook includes steps for creating an image-based watson-machine-learning model, creating a subscription, configuring explainability, and finally generating an explanation for a transaction.

It has been adapted for IBM Cloud Pak for Data 2.5; the original version for IBM Cloud is found here https://github.com/pmservice/ai-openscale-tutorials/blob/master/notebooks/Watson%20OpenScale%20Explanation%20for%20Image%20Multiclass%20Classification%20Model.ipynb 

### Contents
- [1. Setup](#setup)
- [2. Creating and deploying an image-based model](#deployment)
- [3. Subscriptions](#subscription)
- [4. Explainability](#explainability)

<a id="setup"></a>
## 1. Setup

### 1.1 Install Watson OpenScale and WML packages

In [ ]:
!pip install --upgrade ibm-ai-openscale --no-cache | tail -n 1

In [ ]:
!pip install --upgrade watson-machine-learning-client --no-cache | tail -n 1

Note: Restart the kernel to assure the new libraries are being used.

### 1.2 Configure credentials

For Watson OpenScale: Add URL from your IBM Cloud Pak for Data instance, and user name and password that are used for login to the platform. 

<font color='red'>Remember to remove your credentials when sharing the notebook! </font>

In [ ]:
# CP4D specifics: credentials for Watson OpenScale

AIOS_CREDENTIALS = {
    "url": "<url for CP4D>",
    "username": "<username>",
    "password": "<password>"
}

For Watson Machine Learning: Do not change the credentials below! 

In [ ]:
# CP4D specifics: credentials for Watson Machine Learning 

import os
token = os.environ['USER_ACCESS_TOKEN']
WML_CREDENTIALS = {
                   "token": token,
                   "instance_id" : "wml_local",
                   "url": os.environ['RUNTIME_ENV_APSX_URL'],
                   "version": "2.5.0"
                }

<a id="deployment"></a>
## 2. Creating and deploying an image-based model

The dataset used is MNIST dataset of handwritten digits. It consists of 60,000 28x28 grayscale images of the 10 digits, along with a test set of 10,000 images. More information about the dataset can be found here: https://keras.io/datasets/#mnist-database-of-handwritten-digits

Note: Keras and TensorFlow versions supported by WML are: Keras 2.1.6 with TensorFlow 1.13 backend and Keras 2.2.4 with TensorFlow 1.14 backend. The latter combination is used in this notebook.

### 2.1 Creating a model

In [ ]:
!pip install keras==2.2.4
!pip install tensorflow==1.14.0
!pip install keras_sequential_ascii

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras_sequential_ascii import sequential_model_to_ascii_printout
from keras import backend as keras_backend
print(keras.__version__)

In [ ]:
batch_size = 128
num_classes = 10
epochs = 5

In [ ]:
# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if keras_backend.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

In [ ]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# Define Model

def base_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model

In [ ]:
cnn_n = base_model()
cnn_n.summary()

In [ ]:
# Vizualizing model structure
sequential_model_to_ascii_printout(cnn_n)

In [ ]:
# Fit model
print(y_train.shape)
cnn = cnn_n.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test))

In [ ]:
scores = cnn_n.evaluate(x_test, y_test, verbose=0)
print(scores)
print("Accuracy: %.2f%%" % (scores[1]*100))

### 2.2 Storing the model

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

wml_client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)
cnn_n.save("mnist_cnn.h5")
!rm mnist_cnn.tar*
!tar -czvf mnist_cnn.tar.gz mnist_cnn.h5

In [ ]:
!rm mnist_cnn.h5

Add the name of an existing deployment space or a name for a new space to be created: 

In [ ]:
# CP4D specifics: deployment space

DEFAULT_DEPLOYMENT_SPACE="<deployment space name>"

filtered_spaces=[ space['metadata']['guid'] for space in wml_client.spaces.get_details()['resources'] if space['entity']['name']== DEFAULT_DEPLOYMENT_SPACE]
if len(filtered_spaces)>0:
    space_id=filtered_spaces[0]
else:
    space_metadata = {
        wml_client.spaces.ConfigurationMetaNames.NAME: DEFAULT_DEPLOYMENT_SPACE,
        wml_client.spaces.ConfigurationMetaNames.DESCRIPTION: DEFAULT_DEPLOYMENT_SPACE,
    }
    space_details=wml_client.spaces.store(meta_props=space_metadata);
    space_id=space_details['metadata']['guid']
    
print(space_id)

# setting default space
wml_client.set.default_space(space_id)

Make sure that the existing or new deployment space contains the collaborator "admin", which is required for the Watson OpenScale integration later on. Refer to https://www.ibm.com/support/producthub/icpdata/docs/content/SSQNUZ_current/wsj/analyze-data/collaborator-permissions-wml.html on how to add collaborators. 

In [ ]:
# CP4D specifics: project integration

from project_lib import Project
project = Project.access()
project_id=project.get_metadata()['metadata']['guid']

In [ ]:
# CP4D specifics: ModelMetaNames properties, see https://www.ibm.com/support/producthub/icpdata/docs/content/SSQNUZ_current/wsj/wmls/wmls-deploy-python-types.html 

model_name = "MNIST Model"
model_meta = {
    wml_client.repository.ModelMetaNames.NAME: model_name,
    wml_client.repository.ModelMetaNames.RUNTIME_UID: "tensorflow_1.14-py3.6",
    wml_client.repository.ModelMetaNames.TYPE: "keras_2.2.4"
}

In [ ]:
published_model_details = wml_client.repository.store_model(model='mnist_cnn.tar.gz', meta_props=model_meta)

In [ ]:
model_uid = wml_client.repository.get_model_uid(published_model_details)
model_uid

### 2.3 Deploying the model

In [ ]:
deploy_meta = {
     wml_client.deployments.ConfigurationMetaNames.NAME: "MNIST deploy",
     wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
 }
deployment = wml_client.deployments.create(model_uid, meta_props=deploy_meta)

# CP4D specifics: storing the deployment_uid for later
deployment_uid = wml_client.deployments.get_uid(deployment)

In [ ]:
# CP4D specifics: scoring URL(href)
for deployment in  wml_client.deployments.get_details()['resources']:
    print(deployment['entity']['name'],'\t',wml_client.deployments.get_scoring_href(deployment))

## 3. Subscriptions

### 3.1 Configuring OpenScale

In [ ]:
#CP4D specifics: APIClient4ICP
from ibm_ai_openscale import APIClient4ICP
from ibm_ai_openscale.engines import WatsonMachineLearningAsset

aios_client = APIClient4ICP(AIOS_CREDENTIALS)
aios_client.version

### 3.2 Subscribe the asset

In [ ]:
from ibm_ai_openscale.supporting_classes import *

subscription = aios_client.data_mart.subscriptions.add(WatsonMachineLearningAsset(
    model_uid,
    problem_type=ProblemType.MULTICLASS_CLASSIFICATION,
    input_data_type=InputDataType.UNSTRUCTURED_IMAGE,
    probability_column='probability'
))


In [ ]:
aios_client.data_mart.subscriptions.list()

In [ ]:
subscription.get_details()

### 3.3 Score the model and get transaction-id

In [ ]:
!pip install numpy
!pip install matplotlib

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline 
img = np.array(x_test[999], dtype='float')
pixels = img.reshape((28, 28))
plt.imshow(pixels, cmap='gray')
plt.show()

In [ ]:
# CP4D specifics: calling the endpoint, see https://www.ibm.com/support/producthub/icpdata/docs/content/SSQNUZ_current/wsj/wmls/wmls-deploy-python.html
job_payload = {
   wml_client.deployments.ScoringMetaNames.INPUT_DATA: [{
     'values': [x_test[990].tolist()]
   }]
 }

job_details = wml_client.deployments.score(deployment_uid,
                                        job_payload)
print(job_details)

Note: Please wait for a few seconds before running the cell below.

In [ ]:
transaction_id = subscription.payload_logging.get_table_content().scoring_id[0]
print(transaction_id)

<a id="explainability"></a>
## 4. Explainability

### 4.1 Configure Explainability

In [ ]:
subscription.explainability.enable()

In [ ]:
subscription.explainability.get_details()

### 4.2 Get explanation for the transaction

In [ ]:
explanation = subscription.explainability.run(transaction_id, background_mode=False)

In [ ]:
explanation

### The explanation images can be obtained using the cells below

In [ ]:
!pip install Pillow
from PIL import Image
import base64
import io

img = explanation["entity"]["predictions"][0]["explanation"][0]["full_image"]
img_data = base64.b64decode(img)
Image.open(io.BytesIO(img_data))

In [ ]:
img = explanation["entity"]["predictions"][1]["explanation"][0]["full_image"]
img_data = base64.b64decode(img)
Image.open(io.BytesIO(img_data))